In [1]:
from sys import modules

IN_COLAB = 'google.colab' in modules
if IN_COLAB:
    !pip install -q ir_axioms[examples] python-terrier

In [2]:
# Start/initialize PyTerrier.
from pyterrier import started, init

if not started():
    init(tqdm="auto")

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
edition = 28
track = "deep.documents"
dataset_name = "msmarco-document/trec-dl-2019/judged"
contents_field = "body"
depth = 10

In [4]:
from pyterrier.datasets import get_dataset
from ir_datasets import load

dataset = get_dataset(f"irds:{dataset_name}")
ir_dataset = load(dataset_name)

In [5]:
from pathlib import Path

cache_dir = Path("cache/")
index_dir = cache_dir / "indices" / dataset_name.replace("/", "-")
result_dir = Path(
    "/mnt/ceph/storage/data-in-progress/data-research/"
    "web-search/web-search-trec/trec-system-runs"
) / f"trec{edition}" / track
result_files = list(result_dir.iterdir())

In [7]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(
        dataset.get_corpus_iter(),
        fields=[contents_field]
    )

msmarco-document/trec-dl-2019/judged documents:   0%|          | 0/3213835 [00:00<?, ?it/s]

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] If you have a local copy of https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz, you can symlink it here to avoid downloading it again: /home/heinrich/.ir_datasets/downloads/d4863e4f342982b51b9a8fc668b2d0c0
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz

https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz: 0.0%| 0.00/8.50G [00:00<?, ?B/s]
https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz: 0.0%| 16.4k/8.50G [00:00<27:10:14, 86.9kB/s]
https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz: 0.0%| 49.2k/8.50G [00:00<18:47:43, 126kB/s] 
https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz: 0.0%| 98.3k/8.50G [00:00<14:18:39, 165kB/s]
https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz: 0.0%| 164k/8.5

KeyboardInterrupt: 

In [6]:
from pyterrier.io import read_results
from pyterrier import Transformer
from tqdm.auto import tqdm

results = [
    Transformer.from_df(read_results(result_file))
    for result_file in tqdm(result_files, desc="Load results")
]
results_names = [result_file.stem.replace("input.", "") for result_file in result_files]

In [ ]:
from ir_axioms.axiom import (
    ArgUC, QTArg, QTPArg, aSL, PROX1, PROX2, PROX3, PROX4, PROX5, TFC1, TFC3, RS_TF, RS_TF_IDF, RS_BM25, RS_PL2, RS_QL,
    AND, LEN_AND, M_AND, LEN_M_AND, DIV, LEN_DIV, M_TDC, LEN_M_TDC, STMC1, STMC1_f, STMC2, STMC2_f, LNC1, TF_LNC, LB1,
    REG, ANTI_REG, ASPECT_REG, REG_f, ANTI_REG_f, ASPECT_REG_f
)

axioms = [
    ~ArgUC(),  # Very slow due to network access.
    ~QTArg(),  # Very slow due to network access.
    ~QTPArg(),  # Very slow due to network access.
    ~aSL(),
    ~LNC1(),
    ~TF_LNC(),
    ~LB1(),
    ~PROX1(),
    ~PROX2(),
    ~PROX3(),
    ~PROX4(),
    ~PROX5(),
    ~REG(),
    ~REG_f(),
    ~ANTI_REG(),
    ~ANTI_REG_f(),
    ~ASPECT_REG(),
    ~ASPECT_REG_f(),
    ~AND(),
    ~LEN_AND(),
    ~M_AND(),
    ~LEN_M_AND(),
    ~DIV(),
    ~LEN_DIV(),
    ~RS_TF(),
    ~RS_TF_IDF(),
    ~RS_BM25(),
    ~RS_PL2(),
    ~RS_QL(),
    ~TFC1(),
    ~TFC3(),
    ~M_TDC(),
    ~LEN_M_TDC(),
    ~STMC1(),  # Rather slow due many similarity calculations.
    ~STMC1_f(),  # Rather slow due many similarity calculations.
    ~STMC2(),
    ~STMC2_f(),
]
axiom_names = [axiom.axiom.name for axiom in axioms]

In [ ]:
from ir_axioms.backend.pyterrier.experiment import AxiomaticExperiment

experiment = AxiomaticExperiment(
    retrieval_systems=results,
    topics=dataset.get_topics(),
    qrels=dataset.get_qrels(),
    index=index_dir,
    dataset=ir_dataset,
    contents_accessor=contents_field,
    axioms=axioms,
    axiom_names=axiom_names,
    depth=depth,
    filter_by_qrels=False,
    filter_by_topics=False,
    verbose=True,
    cache_dir=cache_dir,
)

In [ ]:
preferences = experiment.preferences

In [ ]:
preferences.to_csv(f"trec-{edition}-{track}-preferences-all-axioms-depth-{depth}.csv")